In [1]:
import os

import dotenv

dotenv.load_dotenv()
HF_ACCESS_TOKEN = os.getenv("HF_ACCESS_TOKEN")
ROOT_PATH = os.path.expanduser(os.getenv("ROOT_PATH"))
ROOT_PATH

'/home/yuva/dev/LLM_Examples'

# Build HF embedding

In [2]:
from langchain_huggingface import HuggingFaceEmbeddings

# https://huggingface.co/spaces/mteb/leaderboard
sentence_transformer_models = [
    "sentence-transformers/all-MiniLM-L6-v2",
    "lier007/xiaobu-embedding-v2",  # rank 1 in chinese
    "Alibaba-NLP/gte-large-en-v1.5",  # rank 21 in english
    # "iampanda/zpoint_large_embedding_zh", # rank 4 in chinese
    # "dunzhang/stella_en_400M_v5", # rank 6 in english  (deprecated)
]

sentence_transformer_model = sentence_transformer_models[1]

hf_embeddings_model = HuggingFaceEmbeddings(
    model_name=sentence_transformer_model,
    cache_folder=os.path.join(ROOT_PATH, "sentence_transformer_model"),
    model_kwargs={"trust_remote_code": True},
)


/home/yuva/dev/LLM_Examples/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# Build HF vector database

In [3]:
import re

from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Qdrant
from langchain_text_splitters import (
    MarkdownHeaderTextSplitter,
    RecursiveCharacterTextSplitter,
)

document_root_path = os.path.join(ROOT_PATH, "docs")
documents = [
    "CNS16190-zh_TW.md",  # 0
    "CNS16190-zh_TW_only_provision.md",  # 1
    "CNS16190-zh_TW_only_provision.pdf",  # 2
    "ts_103701_only_test_scenario.pdf",  # 3
    "ts_103701_only_test_scenario.md",  # 4
    "en_303645_only_provision.pdf",  # 5
]
document_idx = 4  # <-- only change this

chunk_size = 1000
chunk_overlap = 200
model_alias = re.split("[-_]", re.split("/", sentence_transformer_model)[-1])[0]
embedding_cache_path = os.path.join(ROOT_PATH, "embedding_cache")

mode = documents[document_idx].split(".")[-1]
db_collection_names = [
    f"CNS16190_{mode}_hf_{model_alias}_emb",
    f"TS103701_{mode}_hf_{model_alias}_emb",
    f"EN303645_{mode}_hf_{model_alias}_emb",
]
db_collection_idx = db_collection_idx = next(
    (
        idx
        for idx, item in enumerate(db_collection_names)
        if item[:2].casefold() == documents[document_idx][:2].casefold()
    ),
    -1,
)

if os.path.isdir(
    os.path.join(
        embedding_cache_path, "collection", db_collection_names[db_collection_idx]
    )
):
    # database already exists, load it
    hf_vectorstore = Qdrant.from_existing_collection(
        embedding=hf_embeddings_model,
        path=embedding_cache_path,
        collection_name=db_collection_names[db_collection_idx],
    )
else:
    # database does not exist, create it
    try:
        if mode == "md":
            splits = None
            with open(
                os.path.join(document_root_path, documents[document_idx]), "r"
            ) as f:
                markdown_document = f.read()

                headers_to_split_on = [
                    ("#", "Header 1"),
                    ("##", "Header 2"),
                    ("###", "Header 3"),
                    ("####", "Header 4"),
                ]

                markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)
                splits = markdown_splitter.split_text(markdown_document)

        elif mode == "pdf":
            pdf_loader = PyPDFLoader(
                os.path.join(document_root_path, documents[document_idx])
            )
            pdf_doc = pdf_loader.load()
            pdf_text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=chunk_size, chunk_overlap=chunk_overlap
            )
            splits = pdf_text_splitter.split_documents(documents=pdf_doc)

    except Exception as e:
        print(f"An error occurred when the vectorstore createing: {e}")

    finally:
        hf_vectorstore = Qdrant.from_documents(
            splits,
            embedding=hf_embeddings_model,
            path=embedding_cache_path,
            collection_name=db_collection_names[db_collection_idx],
        )

### MarkdownHeaderTextSplitter (for testing)

In [4]:
import os

import dotenv
from langchain_text_splitters import MarkdownHeaderTextSplitter

dotenv.load_dotenv()
ROOT_PATH = os.path.expanduser(os.getenv("ROOT_PATH"))

document_root_path = os.path.join(ROOT_PATH, "docs")
md_header_splits = None
with open(
    os.path.join(document_root_path, "ts_103701_only_test_scenario.md"), "r"
) as f:
    markdown_document = f.read()

    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
        ("####", "Header 4"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)
    md_header_splits = markdown_splitter.split_text(markdown_document)

len(md_header_splits), md_header_splits

(392,
 [Document(metadata={'Header 1': '5.0 TSO 4: Reporting implementation', 'Header 2': '5.0.1 Test group 4-1', 'Header 3': '5.0.1.0 Test group objective'}, page_content='The test group addresses the provision 4-1.'),
  Document(metadata={'Header 1': '5.0 TSO 4: Reporting implementation', 'Header 2': '5.0.1 Test group 4-1', 'Header 3': '5.0.1.1 Test Case 4-1-1 (Conceptual)', 'Header 4': 'Test purpose'}, page_content='The purpose of this test case is the conceptual assessment of the justifications for recommendations that are considered to be not applicable for or not fulfilled by the DUT.'),
  Document(metadata={'Header 1': '5.0 TSO 4: Reporting implementation', 'Header 2': '5.0.1 Test group 4-1', 'Header 3': '5.0.1.1 Test Case 4-1-1 (Conceptual)', 'Header 4': 'Test units'}, page_content='a) The TL **shall** check whether a justification is given in the ICS for each recommendation that is considered to be not applicable for or not fulfilled by the DUT.'),
  Document(metadata={'Header

### UnstructuredMarkdownLoader with element (for testing)

In [5]:
import os

import dotenv
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

dotenv.load_dotenv()
ROOT_PATH = os.path.expanduser(os.getenv("ROOT_PATH"))

document_root_path = os.path.join(ROOT_PATH, "docs")

# database does not exist, create it
loader = UnstructuredMarkdownLoader(
    os.path.join(document_root_path, "ts_103701_only_test_scenario.md"), mode="elements"
)
doc = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(doc)
splits

[Document(metadata={'source': '/home/yuva/dev/LLM_Examples/docs/ts_103701_only_test_scenario.md', 'category_depth': 0, 'last_modified': '2024-09-21T02:18:19', 'languages': ['eng'], 'filetype': 'text/markdown', 'file_directory': '/home/yuva/dev/LLM_Examples/docs', 'filename': 'ts_103701_only_test_scenario.md', 'category': 'Title', 'element_id': '3eb24cc2acf37c1b0a9f4fbda123aca1'}, page_content='5.0 TSO 4: Reporting implementation'),
 Document(metadata={'source': '/home/yuva/dev/LLM_Examples/docs/ts_103701_only_test_scenario.md', 'category_depth': 1, 'last_modified': '2024-09-21T02:18:19', 'languages': ['eng'], 'parent_id': '3eb24cc2acf37c1b0a9f4fbda123aca1', 'filetype': 'text/markdown', 'file_directory': '/home/yuva/dev/LLM_Examples/docs', 'filename': 'ts_103701_only_test_scenario.md', 'category': 'Title', 'element_id': 'c36d324d896d5ddb4a4c1b32429f4450'}, page_content='5.0.1 Test group 4-1'),
 Document(metadata={'source': '/home/yuva/dev/LLM_Examples/docs/ts_103701_only_test_scenario.m

# Similarity search 

In [6]:
import pandas as pd

df = pd.read_csv("../docs/EN303645問題彙整(verdict)_台科大.csv")
df


,provision,status,support,detail,Test Scenario,answer,reason,result,解釋,criteria
0,5.1-1,MC(1),NaN,如果使用密碼並且處於出廠預設設置以外的任何狀態，則所有消費者物聯網設備密碼應為每個設備獨有的...,問題：當使用密碼且處於出廠預設值（例如「admin」）以外的任何狀態時，所有消費者物聯網裝置...,使用者密碼透過wizard由使用者設定且與MAC Address/SN無相關性，且無法反算或...,NaN,NaN,密碼是如何建立的,The verdict PASS is assigned if: • all ...
1,5.1-2,MC(2),NaN,如果每台設備使用預安裝的密碼是獨有的，則應使用一種機制生成這些密碼，以降低對某種或某類設備類...,預先安裝的每台設備的唯一密碼應透過一種機制生成，該機制可降低針對某一類或類型設備的自動攻擊的...,工廠生產線上的工具為每個設備隨機產生一個密碼，將其寫入設備閃存，然後列印在標籤上，讓使用者開...,NaN,NaN,預設密碼的產生機制是什麼,The verdict PASS is assigned if: • a me...
2,5.1-3,M,NaN,用以對裝置鑑別使用者之鑑別機制，應使用對技術、風險及使用的性質適切之最佳實務密碼學。,身份驗證資料在使用者和裝置之間傳輸時應進行加密。\n注意：傳輸通常可以在行動應用程式和裝置之...,本機驗證透過 HMAC-SHA256 質詢使用使用者設定的密碼完成。 \n透過 HTTPs ...,NaN,NaN,身份驗證資料在傳輸過程中如何進行加密保護,The verdict PASS is assigned if: • the ...
3,5.1-4,MC(8),NaN,當使用者可以針對設備進行身份驗證時，設備應向使用者或管理員提供簡單的機制來更改所使用的身份驗證值。,（範例：密碼或指紋類型的令牌）。\n\n問題：\n1. 如果設備允許使用者認證，認證值tok...,1. 登入裝置的裝置密碼，可以在app/web介面修改，修改前必須經過管理會話身分認證。\n...,NaN,NaN,身份驗證後，驗證值(密碼、token..等)應有簡單的機制來更改所使用的身份驗證值,The verdict PASS is assigned if: • a si...
4,5.1-5,MC(5),NaN,當裝置係非受限制時，其應具可用之機制，使得經由網路介面暴力攻擊鑑別機制為不切實際。,當一個設備具有物理上的限制時，可能在數據處理能力、數據通信能力、數據存儲能力或與用戶交互能力...,本地認證機制在連續5次錯誤登錄嘗試後鎖定API訪問60秒，這大大延長了暴力破解攻擊的時間。,NaN,NaN,當裝置是非受限時，要可以防止暴力破解,The verdict PASS is assigned if: • the ...
...,...,...,...,...,...,...,...,...,...,...
62,6.1,M,NaN,製造商應向消費者提供清晰透明的資訊，說明每個設備和服務所處理的個人資料內容、使用方式、處理者...,許多消費者物聯網設備都會處理個人資料。 預計製造商將在消費性物聯網設備中提供支援保護此類個人...,設備本身並無存儲任何除了登入帳號或連線上網之外的任何資訊,NaN,NaN,製造商應向消費者說明每個設備和服務處理的個人資料內容、使用方式和目的，包括參與的第三方。,The verdict PASS is assigned if: • the ...
63,6.2,MC(7),NaN,處理個人資料之依據係消費者的同意，應以有效方式取得此同意。,注意：「以有效方式」取得同意通常涉及讓消費者自由、明顯和明確地選擇其個人資料是否可用於特定目...,NaN,NaN,NaN,處理個人資料需取得消費者同意，並以有效方式取得。,The verdict PASS is assigned if: • obta...
64,6.3,M,NaN,同意處理其個人資料的消費者有權隨時撤回其個人資料。,注意：消費者希望能夠透過適當配置物聯網設備和服務功能來保護自己的隱私。問題：1. 同意處理其...,D-Link 除了登入的帳號密碼之外，不會存儲有關個人隱私資料。,NaN,NaN,消費者有權隨時撤回其個人資料處理的同意。,The verdict PASS is assigned if: • the ...
65,6.4,RC(6),NaN,若由消費者 IoT 裝置及服務蒐集遙測資料，則宜將個人資料之處理保持於預期功能性所必要的最低限度。,問題：\n如果遙測資料是從消費者物聯網設備和服務收集的：\n1. 個人資料的處理是否會維持在...,D-Link 除了登入的帳號密碼之外，不會存儲有關個人隱私資料。,NaN,NaN,處理個人資料應保持在最小必要範圍內。,The verdict PASS is assigned if: • the ...


### One row test

In [7]:
row = df.iloc[10]
query = f"控制措施{row['provision']}，detail:{row['detail']}"
question = f"{query}\
    How to determine PASS or FAIL from the Assignment of verdict of Test group {row['provision']}.\
    Output example: The verdict PASS is assigned if:\
    • the publication of software update support period is understandable and comprehensible for a user with limited technical knowledge. \
    The verdict FAIL is assigned otherwise. "

print(f"{row['provision']}\n")

# similarity search
relevant_docs = hf_vectorstore.search(question, search_type="similarity", k=150)

# filter out the verdict
verdicts = []
for doc in relevant_docs:
    print(doc.metadata)
    # if "Assignment of verdict" in metadata
    if "Header 4" in doc.metadata:
        if doc.metadata["Header 4"] == "Assignment of verdict":
            # Header 2 is like 【5.1.1 Test group 5.1-1】, turn it into [5, 1, 1] to compare with the provision
            if re.split(r"[.-]", doc.metadata["Header 2"].split(" ")[-1]) == re.split(
                r"[.-]", row["provision"].strip()
            ):
                verdicts.append(doc)

for verdict in verdicts:
    print(verdict.metadata)
    print("\n")

5.3-3

{'Header 1': '5.3 Tso 5.3: Keep Software Updated', 'Header 2': '5.3.13 Test group 5.3-13', 'Header 3': '5.3.13.1 Test case 5.3-13-1 (conceptual)', 'Header 4': 'Assignment of verdict', '_id': '098da102f4c545368acfa4a52095ca0a', '_collection_name': 'TS103701_md_hf_xiaobu_emb'}
{'Header 1': '5.3 Tso 5.3: Keep Software Updated', 'Header 2': '5.3.4 Test Group 5.3-4', 'Header 3': '5.3.4.1 Test case 5.3-4-1 (conceptual)', 'Header 4': 'Assignment of verdict', '_id': '7960b3005c8a46f9a17db8d0c0a5a35d', '_collection_name': 'TS103701_md_hf_xiaobu_emb'}
{'Header 1': '5.3 Tso 5.3: Keep Software Updated', 'Header 2': '5.3.5 Test Group 5.3-5', 'Header 3': '5.3.5.1 Test case 5.3-5-1 (conceptual)', 'Header 4': 'Assignment of verdict', '_id': '4e0e3bfcbdf945d08c140f13053d78b2', '_collection_name': 'TS103701_md_hf_xiaobu_emb'}
{'Header 1': '5.3 Tso 5.3: Keep Software Updated', 'Header 2': '5.3.12 Test group 5.3-12', 'Header 3': '5.3.12.1 Test case 5.3-12-1 (conceptual)', 'Header 4': 'Assignment of

### Full test

In [8]:
def get_verdictList(df, topK=10):
    verdict_list = []

    # 67 個測項
    for idx in range(len(df)):

        # create a query
        row = df.iloc[idx]
        query = f"控制措施{row['provision']}，detail:{row['detail']}"
        question = f"{query}\
            How to determine PASS or FAIL from the Assignment of verdict of Test group {row['provision']}.\
            Output example: The verdict PASS is assigned if:\
            • the publication of software update support period is understandable and comprehensible for a user with limited technical knowledge. \
            The verdict FAIL is assigned otherwise. "

        # similarity search
        relevant_docs = hf_vectorstore.search(
            question, search_type="similarity", k=topK
        )

        # filter out the verdict
        verdicts = []
        for doc in relevant_docs:
            # print(doc)

            # if "Assignment of verdict" in metadata
            if "Header 4" in doc.metadata:
                if doc.metadata["Header 4"] == "Assignment of verdict":
                    # Header 2 is like 【5.1.1 Test group 5.1-1】, turn it into [5, 1, 1] to compare with the provision
                    if re.split(
                        r"[.-]", doc.metadata["Header 2"].split(" ")[-1]
                    ) == re.split(r"[.-]", row["provision"].strip()):
                        verdicts.append(doc)

        if len(verdicts) > 1:
            verdict = "The verdict PASS is assigned if:  \n"
            for item in verdicts:
                # verdict += f"{item.page_content}\n"
                splits = re.split(r"[\n]", item.page_content)
                for split in splits:
                    if split.startswith("-"):
                        verdict += f"{split}\n"
            verdict += "The verdict FAIL is assigned otherwise."
        elif len(verdicts) == 1:
            verdict = verdicts[0].page_content
        else:
            verdict = "Not found"

        verdict_list.append(verdict)

    return verdict_list


## Test TopK

In [9]:
import pandas as pd

test_df = pd.DataFrame(columns=["topK", "Not found"])

for topK in range(10, 501, 20):
    verdict_list = get_verdictList(df, topK)
    not_found_count = verdict_list.count("Not found")
    new_row = pd.DataFrame({"topK": [topK], "Not found": [not_found_count]})
    test_df = pd.concat([test_df, new_row], ignore_index=True)
test_df

,topK,Not found
0,10,42
1,30,24
2,50,19
3,70,15
4,90,11
5,110,10
6,130,10
7,150,9
8,170,9
9,190,9


In [10]:
get_verdictList(df, 150)

# drop the last row, then append the retrieved verdicts
df = df.iloc[:, :-1]
df['criteria'] = verdict_list
df

,provision,status,support,detail,Test Scenario,answer,reason,result,解釋,criteria
0,5.1-1,MC(1),NaN,如果使用密碼並且處於出廠預設設置以外的任何狀態，則所有消費者物聯網設備密碼應為每個設備獨有的...,問題：當使用密碼且處於出廠預設值（例如「admin」）以外的任何狀態時，所有消費者物聯網裝置...,使用者密碼透過wizard由使用者設定且與MAC Address/SN無相關性，且無法反算或...,NaN,NaN,密碼是如何建立的,The verdict PASS is assigned if: \n- each pas...
1,5.1-2,MC(2),NaN,如果每台設備使用預安裝的密碼是獨有的，則應使用一種機制生成這些密碼，以降低對某種或某類設備類...,預先安裝的每台設備的唯一密碼應透過一種機制生成，該機制可降低針對某一類或類型設備的自動攻擊的...,工廠生產線上的工具為每個設備隨機產生一個密碼，將其寫入設備閃存，然後列印在標籤上，讓使用者開...,NaN,NaN,預設密碼的產生機制是什麼,The verdict PASS is assigned if: \n- no obvio...
2,5.1-3,M,NaN,用以對裝置鑑別使用者之鑑別機制，應使用對技術、風險及使用的性質適切之最佳實務密碼學。,身份驗證資料在使用者和裝置之間傳輸時應進行加密。\n注意：傳輸通常可以在行動應用程式和裝置之...,本機驗證透過 HMAC-SHA256 質詢使用使用者設定的密碼完成。 \n透過 HTTPs ...,NaN,NaN,身份驗證資料在傳輸過程中如何進行加密保護,The verdict PASS is assigned if: \n- the secu...
3,5.1-4,MC(8),NaN,當使用者可以針對設備進行身份驗證時，設備應向使用者或管理員提供簡單的機制來更改所使用的身份驗證值。,（範例：密碼或指紋類型的令牌）。\n\n問題：\n1. 如果設備允許使用者認證，認證值tok...,1. 登入裝置的裝置密碼，可以在app/web介面修改，修改前必須經過管理會話身分認證。\n...,NaN,NaN,身份驗證後，驗證值(密碼、token..等)應有簡單的機制來更改所使用的身份驗證值,The verdict PASS is assigned if: \n- all mech...
4,5.1-5,MC(5),NaN,當裝置係非受限制時，其應具可用之機制，使得經由網路介面暴力攻擊鑑別機制為不切實際。,當一個設備具有物理上的限制時，可能在數據處理能力、數據通信能力、數據存儲能力或與用戶交互能力...,本地認證機制在連續5次錯誤登錄嘗試後鎖定API訪問60秒，這大大延長了暴力破解攻擊的時間。,NaN,NaN,當裝置是非受限時，要可以防止暴力破解,The verdict PASS is assigned if: \n- the docu...
...,...,...,...,...,...,...,...,...,...,...
62,6.1,M,NaN,製造商應向消費者提供清晰透明的資訊，說明每個設備和服務所處理的個人資料內容、使用方式、處理者...,許多消費者物聯網設備都會處理個人資料。 預計製造商將在消費性物聯網設備中提供支援保護此類個人...,設備本身並無存儲任何除了登入帳號或連線上網之外的任何資訊,NaN,NaN,製造商應向消費者說明每個設備和服務處理的個人資料內容、使用方式和目的，包括參與的第三方。,The verdict PASS is assigned if: \n- the info...
63,6.2,MC(7),NaN,處理個人資料之依據係消費者的同意，應以有效方式取得此同意。,注意：「以有效方式」取得同意通常涉及讓消費者自由、明顯和明確地選擇其個人資料是否可用於特定目...,NaN,NaN,NaN,處理個人資料需取得消費者同意，並以有效方式取得。,The verdict PASS is assigned if: \n- the way ...
64,6.3,M,NaN,同意處理其個人資料的消費者有權隨時撤回其個人資料。,注意：消費者希望能夠透過適當配置物聯網設備和服務功能來保護自己的隱私。問題：1. 同意處理其...,D-Link 除了登入的帳號密碼之外，不會存儲有關個人隱私資料。,NaN,NaN,消費者有權隨時撤回其個人資料處理的同意。,The verdict PASS is assigned if for each categ...
65,6.4,RC(6),NaN,若由消費者 IoT 裝置及服務蒐集遙測資料，則宜將個人資料之處理保持於預期功能性所必要的最低限度。,問題：\n如果遙測資料是從消費者物聯網設備和服務收集的：\n1. 個人資料的處理是否會維持在...,D-Link 除了登入的帳號密碼之外，不會存儲有關個人隱私資料。,NaN,NaN,處理個人資料應保持在最小必要範圍內。,The verdict PASS is assigned if for each telem...


In [11]:
df.to_csv("../docs/test_search_verdicts_with_qdrant.csv", index=False, encoding="utf-8-sig")